In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [5]:
downloadurl = r'http://mis.nyiso.com/public/P-2Alist.htm'
x = simple_get(downloadurl)

In [7]:
# file-output.py
f = open('nyisolmp.html','w')
f.write(x)
f.close()


In [29]:
from bs4 import BeautifulSoup
raw_html = open('nyisolmp.html').read()
html = BeautifulSoup(raw_html, 'html.parser')
listofzips= []
headlink = 'http://mis.nyiso.com/public/'
for a in html.select('a'):
    if a['href'] and a['href'].split('.')[1]=='zip' and a['href'].split('/')[0]=='csv':
        listofzips.append(headlink+a['href'])


In [30]:
f = open('listoflinks.txt','w')
for l in listofzips:
    f.write(l)
    f.write(',\n')
f.close()

    

In [32]:
listofzips[0]

u'http://mis.nyiso.com/public/csv/damlbmp/20181001damlbmp_zone_csv.zip'

In [34]:
# Get the zip file and extract it. 

import requests, zipfile, StringIO
r = requests.get(listofzips[0], stream=True)
z = zipfile.ZipFile(StringIO.StringIO(r.content))
z.extractall()

In [4]:
from google.cloud import storage